If we assume our data to be gaussian , we end up with 


$P(y|x) = \frac{1}{1 + e^{-zy}}$ where $z = w^T x$ and $y \in \{0, 1\}$


Key idea:

Instead of modeling our data with distributions , calculate it and then find the decision boundary find the best fit directly.In other words instead of gettting p(x|y) and p(y) we get p(y|x) directly.











In [1]:
from IPython.display import Image, display

# Display logistic regression visualization images
display(Image(filename='images/logisticregression1.png'))
display(Image(filename='images/logisticregression2.png'))


FileNotFoundError: [Errno 2] No such file or directory: 'logisticregression1.png'